In [39]:
import os

data_dir = "../raw"
datasets = os.listdir(data_dir)

In [80]:
import re
def preprocess_text(text):
    # text = text.replace('-'," - ")
    # text = text.replace('\''," ' ")
    for symbol in ['-','\'','(',')','/',',','+','"']:
        text = text.replace(symbol,f" {symbol} ")
    text = re.sub(r"\s+",' ',text).strip()
    return text

In [81]:
polarity_dict = {
    "positive" : "POS",
    "negative" : "NEG",
    "neutral" : "NEU"
}
def transform_to_num_target(text,targets):
    result_num_targets = []
    splitted_text = text.split()
    for target in targets:
        aspect_term, aspect_category, polarity = target
        aspect_term = preprocess_text(aspect_term)
        splitted_aspect_term = aspect_term.split()
        resulting_index = []
        if aspect_term != "NULL":
            for i in range(0,len(splitted_text)-len(splitted_aspect_term)+1):
                window = splitted_text[i:i+len(splitted_aspect_term)]
                if ' '.join(window) == aspect_term:
                    resulting_index = [i+j for j in range(len(splitted_aspect_term))]
                    break
        else:
            resulting_index = [-1]
        try:
            polarity = polarity_dict[polarity]
        except Exception as e:
            print(polarity)
            raise e
        if len(resulting_index) == 0:
            raise Exception(f"Resulting index cannot be blank, text : {text} | target : {target}")
        result_num_targets.append((resulting_index,aspect_category,polarity))
    return result_num_targets

In [82]:
sample = "The wait staff is very courteous and accomodating .####[('wait staff', 'service general', 'positive')]"
text, targets = sample.split("####")
targets = eval(targets)

In [83]:
transform_to_num_target(text,targets)

[([1, 2], 'service general', 'POS')]

In [84]:
preprocess_text("Food-awesome")

'Food - awesome'

In [86]:
output_dir = "../interim"
data_split = ["train.txt","dev.txt","test.txt"]
for ds in datasets:
    for split in data_split:
        result_data = []
        path = os.path.join(data_dir,ds,split)
        print(path)
        output_path = os.path.join(output_dir,ds)
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        with open(path,'r') as reader:
            data = reader.read().strip().split('\n')
        for line in data:
            try:
                text, targets = line.split("####")
            except Exception as e:
                print(line)
                raise e
            text = preprocess_text(text)
            targets = eval(targets)
            targets = transform_to_num_target(text,targets)
            result_data.append(f"{text}####{targets}")
        with open(os.path.join(output_path,split),'w') as writer:
            for line in result_data:
                writer.write(line + '\n')

../raw\rest15\train.txt
../raw\rest15\dev.txt
../raw\rest15\test.txt
../raw\rest16\train.txt
../raw\rest16\dev.txt
../raw\rest16\test.txt
